In [1]:
import pandas as pd
import pickle
import spacy
import re
import numpy as np

from time import time


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
nlp = spacy.load("en")

suffixes = list(nlp.Defaults.suffixes)
suffixes.remove("#")
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search

## **Echantillonage Train/Test**

In [83]:
X = pickle.load(open("Data/X.pickle", "rb"))
y = pickle.load(open("Data/y.pickle", "rb"))

In [84]:
X_train = X[:20000]
X_test = X[20000:]

y_train = y[:20000]
y_test = y[20000:]

# 1)- **Nettoyage**

### **Fonction Nettoyage**

On va faire une fonction à base de regex qui enlèvera les chiffres, etc..

In [68]:
def clean_text(texte):
    
    char_gardes = r'[^a-z#+.\s]'
    return re.sub(char_gardes, '', texte)   

### **Fonction stopWords + Verbes + Space**

La suppression des stopwords n'est pas systématique en NLP. Mais dans notre cas, où on travaille sur l'extraction de thème, ça l'est.

In [100]:
def remove_sw(texte, return_verbs, return_token):
    
    doc = nlp(texte)
    
    if return_verbs :
    
        tokens = [token for token in doc if \
                  token.is_stop == False and token.is_punct == False\
                  and token.is_space == False
                 ]
    
    else :
        
        tokens = [token for token in doc if \
                  token.is_stop == False and token.is_punct == False\
                  and token.pos_ != "VERB"\
                  and token.is_space == False
                 ]
    
    if return_token :
        
        return tokens
    
    txt = ""
    
    for t in tokens :
        
        txt = txt + " " + t.text
        txt = txt.lstrip()
        
    return txt


def pipe_clsw(texte, return_verbs = True, return_token = True):
    """
    Nettoye et enlève les SW d'un text/str et retourne une liste de tokens.
    Par défaut : 
    - On garde les verbe. Si False on les élimine du résultat.
    - On retourne une liste de token. Si False on retourne une str des tokens.
    """
    
    res = remove_sw(clean_text(texte), return_verbs, return_token)
    
    return res

In [94]:
X_train[250]

"flutter animated container i have a raisedbutton widget and an animatedcontainer widget in a screen, and the idea is that upon pressing the raisedbutton the width of the animatedcontainer would then decrease in a given duration. the documentation of the animatedcontainer states that all i would need to do is declare the width of the widget as a variable, and then setstate(() {}) after changing the value and it will automatically change to that value during the duration. i have tried to implement this and upon pressing the raisedbutton the variables value definitely changes (based on printing the value of it after pressing it), however the widget's width does not change with it. am i missing something obvious? my widgets are within a container in a pageview and my code for the raisedbutton and animatedcontainer is as follows: here is my widget tree:"

In [102]:
res = pipe_clsw(X_train[250])
print(res)

[flutter, animated, container, raisedbutton, widget, animatedcontainer, widget, screen, idea, pressing, raisedbutton, width, animatedcontainer, decrease, given, duration, documentation, animatedcontainer, states, need, declare, width, widget, variable, setstate, changing, value, automatically, change, value, duration, tried, implement, pressing, raisedbutton, variables, value, definitely, changes, based, printing, value, pressing, widgets, width, change, missing, obvious, widgets, container, pageview, code, raisedbutton, animatedcontainer, follows, widget, tree]


In [103]:
res = pipe_clsw(X_train[250], False, False)
res

'flutter container raisedbutton widget animatedcontainer widget screen idea raisedbutton width animatedcontainer duration documentation animatedcontainer width widget variable setstate value automatically value duration raisedbutton variables value definitely changes value widgets width obvious widgets container pageview code raisedbutton animatedcontainer widget tree'

# **Approche non-supervisée**

## **LDA - Count**

### **Texte sans verbes**

In [106]:
X_train_clean = [pipe_clsw(t, False, False) for t in X_train]
X_train_clean[:3]

['script inside aws lambda function bash script inside lambda function aws docs code python nodejs java documents possible bash concrete evidence example',
 'boot controller content negotiation simple rest controller springboot application sure content negotiation json xml contenttype parameter request header wrong controller method json method contenttype applicationxml textxml methods different mapping different content type able xml xml mediatypes single method like example message endpoint xml contenttype request applicationxmljson contenttype applicationjson help editi controller media types',
 'error goto statement vba code particular value excel sheet ctrl+f command code message problem error message message box error']

In [107]:
pickle_out = open("Data/dat_clean_text_no_verbs.pickle", "wb")
pickle.dump(X_train_clean, pickle_out)
pickle_out.close()

**CountVectorizer**

In [56]:
X_train_clean = pickle.load(open("Data/dat_clean_text_no_verbs.pickle", "rb"))

# HP du nombre de sujets : 31 pour les 30 suejts représentés par les tags + 1 "misc"
nb_sujets = 31

In [57]:
# bypass tokenizer à cause de node.js on veut une separation que sur les espace, ce qui permet de conserver des
# expressions contenant des points qui nous paraiossent importantes...

def neutral_tokenizer(tokens):
    
    return tokens.split(" ")

tf_vect = CountVectorizer(max_df = 0.95, min_df = 4, tokenizer = neutral_tokenizer)
tf_vect.fit(X_train_clean)

CountVectorizer(max_df=0.95, min_df=4,
                tokenizer=<function neutral_tokenizer at 0x0000000014AAE9D0>)

In [58]:
X_train_clean_vect = tf_vect.transform(X_train_clean)
X_train_clean_vect.shape

(20000, 6722)

In [71]:
lda_tf = LatentDirichletAllocation(n_components = nb_sujets, 
                                max_iter = 5,
                                learning_method = "online",
                                learning_offset = 50., 
                                random_state = 47)

In [72]:
start = time()
lda_tf.fit(X_train_clean_vect)
print(f"L'entrainement de la LDA a mis : {time() - start} secondes.")

L'entrainement de la LDA a mis : 30.283731937408447 secondes.


In [73]:
def montrer_sujets(model, feature_names, nb_top_words) :
    
    for sujet_idx, sujet in enumerate(model.components_):
        print(f"topic numéro {sujet_idx} :")
        print(" ".join(feature_names[i] for i in sujet.argsort()[: -nb_top_words -1:-1]))
        print("----------------------------------")
        
def retourne_tags(model, feature_names) :
    
    liste = []
    
    for sujet_idx, sujet in enumerate(model.components_):
        # print(f"topic numéro {sujet_idx} :")
        
        liste.append(feature_names[sujet.argmax()])
        
    return liste

In [101]:
# countvect token par defaut
montrer_sujets(lda_tf, tf_vect.get_feature_names(), 10)

topic numéro 0 :
api request server service url error response chrome web http
----------------------------------
topic numéro 1 :
function query loop functions code aws promise boolean n inside
----------------------------------
topic numéro 2 :
devtools dag airflow encryption soap wcf bitmap clang fire concurrent
----------------------------------
topic numéro 3 :
error object class nt method code string like property new
----------------------------------
topic numéro 4 :
array number y x size item items numpy c memory
----------------------------------
topic numéro 5 :
version studio visual nt code + database mysql way question
----------------------------------
topic numéro 6 :
angular component element components event template parent page like child
----------------------------------
topic numéro 7 :
npm start node v node.js module package.json error cli ng
----------------------------------
topic numéro 8 :
column dataframe pandas data columns values row rows csv like
---------

Extraction de la liste des meilleurs mots pouvant servir de tags...

In [102]:
liste_tags_lda_tf = retourne_tags(lda_tf, tf_vect.get_feature_names())
print(liste_tags_lda_tf)

['api', 'function', 'devtools', 'error', 'array', 'version', 'angular', 'npm', 'column', 'react', 'db', 'user', 'view', 'file', 'image', 'data', 'table', 'git', 'block', 'package', 'date', 'consumer', 'difference', 'test', 'div', 'attribute', 'python', 'value', 'project', 'android', 'error']


**Transformation**

In [103]:
topic_score_tf = lda_tf.transform(X_train_clean_vect)
topic_score_tf.shape

(20000, 31)

In [104]:
topic_score_tf[0]

array([0.00140252, 0.59202265, 0.00140252, 0.00140252, 0.00140252,
       0.00140252, 0.00140252, 0.04488078, 0.00140252, 0.00140252,
       0.00140252, 0.00140252, 0.00140252, 0.00140252, 0.00140252,
       0.00140252, 0.08904097, 0.08943088, 0.00140252, 0.00140252,
       0.00140252, 0.00140252, 0.00140252, 0.00140252, 0.00140252,
       0.00140252, 0.04804115, 0.05761499, 0.04530798, 0.00140252,
       0.00140252])

In [105]:
topic_predicted_tf = np.argmax(topic_score_tf, axis = 1)

# sauvegarde
pickle_out = open("Data/topic_predicted_tf.pickle", "wb")
pickle.dump(topic_predicted_tf, pickle_out)
pickle_out.close()

topic_predicted_tf

array([ 1, 15, 30, ...,  3, 14, 30], dtype=int64)

LDA a prédit des topics et a assigné les questions à ces topics... Mais comment en tire-t-on des tags pour nos question ?

### **Approche totalement non-supervisée**

Cette approche consisterait à récupérer les 1ers termes du model pour chaque topic et à les choisir pour tags correspondant à ce topic.

Pour évaluer les résultats qu'on pourrait obtenir par cette approche au regard des tags à prévoir il faut regarder deux choses :<br>- Les tags eux-mêmes.<br>- Le nombre de tags par question.

### **Comparaison tags originaux, tags LDA**

In [153]:
print(liste_tags_lda_tf)

['api', 'function', 'devtools', 'error', 'array', 'version', 'angular', 'npm', 'column', 'react', 'db', 'user', 'view', 'file', 'image', 'data', 'table', 'git', 'block', 'package', 'date', 'consumer', 'difference', 'test', 'div', 'attribute', 'python', 'value', 'project', 'android', 'error']


In [154]:
vect_y = CountVectorizer(tokenizer = neutral_tokenizer).fit(y_train)
liste_tags_reels = vect_y.get_feature_names()
print(liste_tags_reels)

['android', 'angular', 'asp.net', 'c#', 'code', 'css', 'docker', 'flutter', 'git', 'google', 'html', 'ios', 'java', 'javascript', 'jquery', 'json', 'laravel', 'misc', 'mysql', 'node.js', 'pandas', 'php', 'python', 'react', 'spring', 'sql', 'swift', 'typescript', 'visual', 'web', 'windows']


Eléments communs :

In [155]:
common = list(set(liste_tags_lda_tf).intersection(liste_tags_reels))
print(common, len(common))

['react', 'android', 'git', 'python', 'angular'] 5


Il n'y a que 5 tags en commun, ce qui jette de sérieux doute sur la pertinence de notre démarche...

### **Nombre de tags par question**

La façon la plus pertinente de soutirer plusieurs tags à notre modèle LDA serait de passer par une logique de seuil. On peut donc regarder s'il y a un rapport éventuel entre le nombre de tags à prévoir pour chaque question et les topics score prédit par le modèle LDA.

In [158]:
# Liste du nombre de tags à prédire de y_train
len_y = []

for s in y_train :
    len_y.append(len(s.split()))
    
len_y[:10]

[1, 2, 1, 2, 1, 1, 1, 2, 1, 2]

Regardons à quels **topic score** correspondent les n-ièmes tags cibles pour chaque question (pour les 10 première questions...)

In [173]:
seuils_topic_score = []

for i, n in zip(topic_score_tf[:10], len_y[:10]) :
    ar = np.sort(i)
    print(ar[-n])    

0.5920226517034007
0.2153120613717825
0.7447321786377393
0.3815435970685162
0.4659271432855029
0.7670075957299393
0.4534244923828475
0.06881730731637903
0.3007856542725834
0.26145307184557964


Rehardons si ces valeurs pourrait se déduire d'une condition type "être au dessus d'un certain seuil"...

In [174]:
for i, n in zip(topic_score_tf[:10], len_y[:10]) :
    ar = np.sort(i)
    print(ar[-(n+1)])

0.08943087847676548
0.19542571477915335
0.09698229220278344
0.1356607296680963
0.22420731721295192
0.09238593551882536
0.3712734850698988
0.002150537634410425
0.21620325705915075
0.0933930027653001


Il semble impossible de définir un seuil de "topic score" à partir duquel tagger une question permettant de faire correspondre un nombre de tags issus du modèle LDA avec le nombre de tags cibles.

**Conclusion**

Les tags déduits du modèles LDA sont différents des tags cibles.<br>Il semble de plus difficile de trouver une logique permettant de faire matcher un nombre de tags par question issu du modèle LDA avec les nombres de tags par question de la cible.<br><br>Ce **manque d'éléments communs** rend **impossible** une **évaluation sérieuse** du modèle.

L'utilisation de modèle LDA est adaptée à la recherche thématique dans un cadre vierge, mais n'est pas pertinente ici, dans un cadre sémantique déjà fixé, qui rend hasardeux l'attribution de tag par des méthodes statistiques non supervisée.

## **Approche LDA - TFIDF**

Pour voir par exemple si avec une telle approche on trouverait plus de tags présents dans la liste cible...

In [201]:
X_train_clean = pickle.load(open("Data/dat_clean_text_no_verbs.pickle", "rb"))
nb_sujets = 31

In [202]:
def neutral_tokenizer(tokens):
    
    return tokens.split(" ")

#idf_vect = TfidfVectorizer(tokenizer = neutral_tokenizer, token_pattern = r'(?u)\b\w\w+__.\([\w\s]*\)')
#tokpat = r'([^a-z#+.\s]{2,})'
token_pattern = tokpat
idf_vect = TfidfVectorizer(max_df = 0.95, min_df = 5)

idf_vect.fit(X_train_clean)

TfidfVectorizer(max_df=0.95, min_df=5)

In [203]:
X_train_clean_idf = idf_vect.transform(X_train_clean)
X_train_clean_idf.shape

(20000, 5777)

In [204]:
lda_idf = LatentDirichletAllocation(n_components = nb_sujets, 
                                max_iter = 5,
                                learning_method = "online",
                                learning_offset = 50., 
                                random_state = 47)

In [205]:
start = time()
lda_idf.fit(X_train_clean_idf)
print(f"L'entrainement de la LDA_idf a mis : {time() - start} secondes.")

L'entrainement de la LDA_idf a mis : 21.74224352836609 secondes.


In [206]:
montrer_sujets(lda_idf, idf_vect.get_feature_names(), 10)

topic numéro 0 :
date format datetime modal timezone yyyymmdd instagram ddmmyyyy epoch mmddyyyy
----------------------------------
topic numéro 1 :
code error nt angular like function data value component array
----------------------------------
topic numéro 2 :
certificate ssl tls gulp nodejs curl websocket pgadmin ip openssl
----------------------------------
topic numéro 3 :
color background px white svg theme black button textinputlayout cat
----------------------------------
topic numéro 4 :
uuid oracle schema arrays ubuntu equal commands contents api open
----------------------------------
topic numéro 5 :
ngoninit async asyncawait babel promise obviously scratch await babelrc angular
----------------------------------
topic numéro 6 :
tensorflow tf cuda gpu placeholder cpu uitextfield duration startdate tensorflowgpu
----------------------------------
topic numéro 7 :
markdown ivy gif other textual esm commonjs symbol es standards
----------------------------------
topic numéro 

In [207]:
topic_score_idf = lda_idf.transform(X_train_clean_idf)
topic_score_idf.shape

(20000, 31)

In [209]:
topic_predicted_idf = np.argmax(topic_score_idf, axis = 1)
topic_predicted_idf

array([ 1,  1,  1, ...,  1, 28, 28], dtype=int64)

In [214]:
topic_score_idf[150]

array([0.00813409, 0.72688735, 0.00813409, 0.00813409, 0.00813409,
       0.00813409, 0.00813409, 0.00813409, 0.00813409, 0.00813409,
       0.00813409, 0.00813409, 0.00813409, 0.03722393, 0.00813409,
       0.00813409, 0.00813409, 0.00813409, 0.00813409, 0.00813409,
       0.00813409, 0.00813409, 0.00813409, 0.00813409, 0.00813409,
       0.00813409, 0.00813409, 0.00813409, 0.00813409, 0.00813409,
       0.00813409])

In [192]:
liste_tags_lda_idf = retourne_tags(lda_idf, idf_vect.get_feature_names())
print(liste_tags_lda_idf)

['date', 'code', 'certificate', 'color', 'uuid', 'ngoninit', 'tensorflow', 'markdown', 'docker', 'folder', 'jupyter', 'kubernetes', 'jenkins', 'ubuntu', 'firestore', 'tid', 'order', 'python', 'android', 'icon', 'swift', 'signalr', 'git', 'xcode', 'latex', 'appreciable', 'checkbox', 'product', 'error', 'django', 'bytecode']


In [193]:
print(liste_tags_reels)

['android', 'angular', 'asp.net', 'c#', 'code', 'css', 'docker', 'flutter', 'git', 'google', 'html', 'ios', 'java', 'javascript', 'jquery', 'json', 'laravel', 'misc', 'mysql', 'node.js', 'pandas', 'php', 'python', 'react', 'spring', 'sql', 'swift', 'typescript', 'visual', 'web', 'windows']


In [194]:
common_idf = list(set(liste_tags_lda_idf).intersection(liste_tags_reels))
print(common_idf, len(common_idf))

['android', 'git', 'python', 'swift', 'code', 'docker'] 6


6 terme en communs, c'est un peu mieux que l'approche **CountVectorizer**, mais rien de transcendant... On ne pousse pas plus loin, on reste sur la même conclusion.

In [197]:
topic_score_idf = lda_idf.transform(X_train_clean_idf)
topic_score_idf.shape

(20000, 31)

In [198]:
topic_predicted_idf = np.argmax(topic_score_idf, axis = 1)

# sauvegarde
pickle_out = open("Data/topic_predicted_idf.pickle", "wb")
pickle.dump(topic_predicted_idf, pickle_out)
pickle_out.close()

topic_predicted_idf

array([ 1,  1,  1, ...,  1, 28, 28], dtype=int64)

In [215]:
topic_predicted_idf[100:200]

array([28, 28, 28,  1,  1, 28, 28,  1, 28, 28,  1, 28, 17,  1, 28, 28,  1,
       28,  1, 28, 28, 28,  1, 28, 28,  1, 28, 28,  1, 28,  1,  0, 28, 28,
       28, 12,  1, 28, 28, 28, 28,  1, 28, 28, 28,  1, 28,  1,  3, 28,  1,
        1,  1, 28,  1,  1,  1,  0, 28,  1, 28,  1, 28,  1,  1, 28,  1,  1,
       28,  1,  1,  1, 28,  1,  1,  1,  1, 28,  1,  1, 28,  1, 18,  1,  1,
        1, 18,  1, 28,  1,  1,  1, 12, 28, 28, 28,  1, 28, 28, 28],
      dtype=int64)

In [217]:
topic_count_idf = pd.Series(topic_predicted_idf)
topic_count_idf.value_counts()

1     10317
28     8629
18      390
17      141
8        88
22       83
23       65
13       49
20       41
0        30
3        28
6        23
29       20
14       17
12       17
10       15
11       12
2        12
19        9
26        7
7         5
27        2
dtype: int64

In [218]:
topic_count_tf = pd.Series(topic_predicted_tf)
topic_count_tf.value_counts()

30    3530
3     2590
13    1390
29    1359
0     1317
27    1191
6     1146
5     1040
28     725
26     684
14     665
8      604
4      479
20     454
1      388
24     374
23     342
16     315
17     233
7      223
12     212
11     203
15     165
9      147
19     110
25      44
22      29
10      16
18      13
2        7
21       5
dtype: int64

# **Approche Mixte**

La LDA a note chaque question au regard de topics qu'elle a détecté, on pourrait se servir de ces notes attribuées comme features dans une approche supervisée multilabels.<br>Dans ce cadre, la LDA apparait comme une étape supplémentaire de processing, en l'occurence de **réduction dimensionnelle**.

Par contre, comme il n'est pas question ici que la LDA extraie elle-même les noms des tags, on va reprendre le pre-processing depuis le début (recours à la lemmatization/stemming) + verbe.

1- Approche données actuelles LDA CV / TFODF<br>
2- Nouveau processing, verbe + lemma + stemming ++++ lda ++++ CV et TFIDF